# AI4Industry - Mensaflow © - RAG avec Ollama + Qdrant
## Architecture: Mistral + Vector DB + Agents (Math + Wikipedia)

**Corrigé par Claude - Janvier 2025**

## 📦 Installation des dépendances

In [ ]:
%%capture
# Installation Ollama
!sudo apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

# Installation des packages Python
!pip install -q langchain==0.3.20 langchain-community==0.3.20 langchain-core==0.3.40
!pip install -q qdrant-client sentence-transformers wikipedia
!pip install -q numexpr sympy

## 🚀 Démarrage du serveur Ollama

In [ ]:
import os
import subprocess
import time

# Démarrer Ollama en arrière-plan
print("🔄 Démarrage du serveur Ollama...")
subprocess.Popen(["ollama", "serve"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
time.sleep(5)

# Télécharger Mistral 7B
print("📥 Téléchargement de Mistral 7B...")
!ollama pull mistral
print("✅ Mistral prêt !")

## 🧠 Initialisation du LLM

In [ ]:
from langchain_community.llms import Ollama

llm = Ollama(
    model="mistral",
    base_url="http://localhost:11434",
    temperature=0.7
)

# Test rapide
response = llm.invoke("Bonjour, qui es-tu ?")
print("🤖 Mistral:", response[:200])

## 🗃️ Configuration de Qdrant (Vector Database)

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Qdrant
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams

# Embedding model
print("📊 Chargement du modèle d'embeddings...")
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'}
)

# Client Qdrant (en mémoire pour Colab)
print("🗄️ Initialisation de Qdrant...")
qdrant_client = QdrantClient(":memory:")

collection_name = "ai4industry_docs"

# Créer la collection
qdrant_client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=384, distance=Distance.COSINE)
)

print("✅ Qdrant prêt !")

## 📚 Alimentation de la base vectorielle (Wikipedia)

In [ ]:
import wikipedia
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

# Configurer Wikipedia en français
wikipedia.set_lang("fr")

# Topics pertinents pour l'industrie
topics = [
    "Intelligence artificielle",
    "Machine learning",
    "Industrie 4.0",
    "Maintenance prédictive",
    "Internet des objets"
]

print("📖 Récupération des articles Wikipedia...")
documents = []

for topic in topics:
    try:
        page = wikipedia.page(topic)
        doc = Document(
            page_content=page.content,
            metadata={"source": topic, "url": page.url}
        )
        documents.append(doc)
        print(f"  ✓ {topic}")
    except Exception as e:
        print(f"  ✗ {topic}: {e}")

# Découpage en chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)
chunks = text_splitter.split_documents(documents)
print(f"\n📝 {len(chunks)} chunks créés")

# Indexation dans Qdrant
print("🔄 Indexation dans Qdrant...")
vectorstore = Qdrant.from_documents(
    chunks,
    embeddings,
    client=qdrant_client,
    collection_name=collection_name
)
print("✅ Base vectorielle prête !")

## 🛠️ Création des outils (Tools)

In [ ]:
from langchain.tools import Tool
from langchain_community.utilities import WikipediaAPIWrapper
import numexpr

# 1. Outil de calcul mathématique
def calculate(expression: str) -> str:
    """Évalue une expression mathématique."""
    try:
        result = numexpr.evaluate(expression).item()
        return f"Résultat: {result}"
    except Exception as e:
        return f"Erreur de calcul: {str(e)}"

math_tool = Tool(
    name="Calculator",
    func=calculate,
    description="Utile pour les calculs mathématiques. Exemple: '2+2' ou '10*5'"
)

# 2. Outil Wikipedia
wikipedia_wrapper = WikipediaAPIWrapper(lang="fr")
wikipedia_tool = Tool(
    name="Wikipedia",
    func=wikipedia_wrapper.run,
    description="Recherche des informations sur Wikipedia en français"
)

# 3. Outil RAG (retrieval)
def rag_search(query: str) -> str:
    """Recherche dans la base vectorielle."""
    docs = vectorstore.similarity_search(query, k=3)
    context = "\n\n".join([doc.page_content for doc in docs])
    return context

rag_tool = Tool(
    name="RAG_Search",
    func=rag_search,
    description="Recherche dans la base de connaissances vectorielle (documents indexés)"
)

tools = [math_tool, wikipedia_tool, rag_tool]
print("✅ 3 outils créés: Calculator, Wikipedia, RAG_Search")

## 🤖 Création de l'Agent (CORRIGÉ)

In [ ]:
# CORRECTION: Import depuis langchain.agents (nouvelle API)
from langchain.agents import create_react_agent, AgentExecutor
from langchain_core.prompts import PromptTemplate

# Template pour l'agent ReAct
template = """Tu es un assistant IA spécialisé dans l'industrie et la technologie.

Tu as accès aux outils suivants:
{tools}

Noms des outils: {tool_names}

Utilise ce format:
Question: la question posée
Thought: je dois réfléchir à quel outil utiliser
Action: le nom de l'outil (un parmi [{tool_names}])
Action Input: l'entrée pour l'outil
Observation: le résultat de l'outil
... (répète Thought/Action/Action Input/Observation si nécessaire)
Thought: je connais maintenant la réponse finale
Final Answer: la réponse finale à la question

Question: {input}
{agent_scratchpad}
"""

prompt = PromptTemplate.from_template(template)

# Créer l'agent
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

# Créer l'executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5
)

print("✅ Agent RAG créé avec succès !")

## 🧪 Tests de l'Agent

In [ ]:
# Test 1: Calcul mathématique
print("="*60)
print("TEST 1: Calcul mathématique")
print("="*60)
response = agent_executor.invoke({"input": "Calcule 147 * 23 + 456"})
print("\n📊 Réponse:", response['output'])

In [ ]:
# Test 2: Recherche Wikipedia
print("\n" + "="*60)
print("TEST 2: Recherche Wikipedia")
print("="*60)
response = agent_executor.invoke({"input": "Qu'est-ce que l'Industrie 4.0 ?"})
print("\n📚 Réponse:", response['output'])

In [ ]:
# Test 3: RAG (recherche vectorielle)
print("\n" + "="*60)
print("TEST 3: RAG Search")
print("="*60)
response = agent_executor.invoke({"input": "Parle-moi du machine learning dans l'industrie"})
print("\n🔍 Réponse:", response['output'])

In [ ]:
# Test 4: Question complexe combinant plusieurs outils
print("\n" + "="*60)
print("TEST 4: Question complexe")
print("="*60)
response = agent_executor.invoke({
    "input": "Si une usine utilise 3 capteurs IoT qui génèrent chacun 500 MB de données par jour, combien de GB cela représente en une semaine ?"
})
print("\n🏭 Réponse:", response['output'])

## 💬 Interface Interactive

In [ ]:
def chat():
    print("\n🤖 Agent RAG AI4Industry - Mensaflow")
    print("Type 'exit' pour quitter\n")
    
    while True:
        user_input = input("Vous: ")
        if user_input.lower() in ['exit', 'quit', 'sortir']:
            print("Au revoir !")
            break
        
        try:
            response = agent_executor.invoke({"input": user_input})
            print(f"\nAgent: {response['output']}\n")
        except Exception as e:
            print(f"Erreur: {str(e)}\n")

# Lancer le chat
chat()

## 📊 Statistiques et Diagnostics

In [ ]:
# Vérifier le nombre de documents dans Qdrant
collection_info = qdrant_client.get_collection(collection_name)
print(f"📈 Nombre de vectors dans Qdrant: {collection_info.vectors_count}")

# Test de similarité
query = "maintenance prédictive"
results = vectorstore.similarity_search(query, k=3)
print(f"\n🔍 Top 3 résultats pour '{query}':")
for i, doc in enumerate(results, 1):
    print(f"\n{i}. Source: {doc.metadata.get('source', 'N/A')}")
    print(f"   Extrait: {doc.page_content[:150]}...")

---
## 🎯 Résumé de l'Architecture

**Stack Technique:**
- **LLM**: Mistral 7B (via Ollama)
- **Vector DB**: Qdrant (in-memory)
- **Embeddings**: sentence-transformers/all-MiniLM-L6-v2
- **Agents**: Calculator, Wikipedia, RAG Search
- **Framework**: LangChain 0.3.x

**Corrections appliquées:**
1. ✅ Import correct de `AgentExecutor` (depuis `langchain.agents`)
2. ✅ Utilisation de `create_react_agent` moderne
3. ✅ Template ReAct compatible
4. ✅ Gestion d'erreurs améliorée

**Mensaflow © 2025 - Formation AI4Industry CNAM**